In [1]:
# Observacao: conflito com o nome `d`
# `d` ja existe como instancia Down(). Se voce quiser usar `d` como indice,
# guarde um alias antes de sobrescrever:
#
# from lyra_geometry import *
# dn = d
# a, b, c, d = st.index('a b c d')
# # use dn(...) quando precisar da antiga instancia `d`


# Lyra

In [3]:
import sys

sys.path.append("../src")

In [4]:
import sympy as sp
import lyra_geometry as pl

# Schwarzschild Space-Time

In [6]:
import sympy as sp
import lyra_geometry as pl

# -- Metrics --------
print("Defining metrics")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)

nu = sp.Function('nu')
lamb = sp.Function('lambda')

g = sp.diag(sp.exp(nu(r)), -sp.exp(lamb(r)), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}

st = pl.SpaceTime(coords=x, metric=g)

st.update()

Definig metrics


In [13]:
a, b, m, n  = st.index("alpha beta mu nu")

st.g[-m, -n]

[[exp(nu(r)), 0, 0, 0], [0, -exp(lambda(r)), 0, 0], [0, 0, -r**2, 0], [0, 0, 0, -r**2*sin(theta)**2]]

In [14]:
st.g[+m, +n]

[[exp(-nu(r)), 0, 0, 0], [0, -exp(-lambda(r)), 0, 0], [0, 0, -1/r**2, 0], [0, 0, 0, -1/(r**2*sin(theta)**2)]]

In [15]:
st.g[+m, +n] * st.g[-n, -a]

In [16]:
st.gamma

In [17]:
st.riemann

In [18]:
st.ricci

In [20]:
st.scalar_curvature

In [27]:
st.einstein.fmt()

In [29]:
Ricc = st.ricci
R = st.scalar_curvature
g = st.g

(Ricc[-m,-n] - sp.Rational(1,2) * g[-m,-n]*R).fmt()

[[-exp(-lambda(r))*exp(nu(r))*Derivative(lambda(r), r)/r - exp(nu(r))/r**2 + exp(-lambda(r))*exp(nu(r))/r**2, 0, 0, 0], [0, -Derivative(nu(r), r)/r + exp(lambda(r))/r**2 - 1/r**2, 0, 0], [0, 0, r**2*exp(-lambda(r))*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*Derivative(nu(r), r)/2, 0], [0, 0, 0, r**2*exp(-lambda(r))*sin(theta)**2*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*sin(theta)**2*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*sin(theta)**2*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*sin(theta)**2*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*sin(theta)**2*Derivative(nu(r), r)/2]]

In [30]:
assert st.einstein.fmt() == (Ricc[-m,-n] - sp.Rational(1,2) * g[-m,-n]*R).fmt()

# FLRW Space-Time

In [6]:
import sympy as sp
import lyra_geometry as pl

print("Definindo metrica")
t, x, y, z = sp.symbols('t x y z', real=True)
a = sp.Function('a')

# -- SpaceTime ----------
st = pl.SpaceTime(
    coords=(t, x, y, z),
    metric=sp.diag(1, -a(t)**2, -a(t)**2, -a(t)**2)
)


# -- Index --------
_, a, b, g, d, e, m, n, l, s, h, k  = st.index("empty alpha beta gamma delta epsilon mu nu lambda sigma eta kappa")


# -- Escala de Lyra ----------
print("Definindo escala")
phi = st.set_scale(sp.Function('phi')(t))

# # -- Tensor de Levi Civita ----------
# print("Definindo LeviCivita")
# eps = st.tensor.from_function(
#     lambda mu, nu, rho, sig: sp.sqrt(-st.detg) * sp.LeviCivita(mu, nu, rho, sig),
#     signature=(pl.D, pl.D, pl.D, pl.D),
#     name="eps",
# )

# # -- Non-Metricity ----------
# print("Definindo Non-Metricity")
# M = st.tensor.from_function(
#     lambda a, b, c: 0,
#     signature=(pl.u, pl.D, pl.D),
#     name="M",
# )
# st.set_nonmetricity(M)

# # -- Torsion ----------
# print("Definindo Torsion")

# def torsion_axial(mu):
#     if mu == 0:
#         return sp.Function('s_t')(r)
#     if mu == 1:
#         return sp.Function('s_r')(r)
#     return 0

# s = st.tensor.from_function(torsion_axial, signature=(pl.U,), name="s")


# def torsion_trace(mu):
#     if mu == 0:
#         return sp.Function('v_t')(r)
#     if mu == 1:
#         return sp.Function('v_r')(r)
#     return 0

# v = st.tensor.from_function(torsion_trace, signature=(pl.D,), name="v")


# def torsion_component(m, n, r_):
#     return (
#         sum(eps(pl.D, pl.D, pl.D, pl.D).comp[m, n, r_, sig] * s(pl.U).comp[sig] for sig in range(st.dim))
#         + sp.Rational(1, 3) * (
#             st.g[m, n] * v(pl.D).comp[r_]
#             - st.g[m, r_] * v(pl.D).comp[n]
#         )
#     )

# tau = st.set_torsion(st.tensor.from_function(torsion_component, signature=(pl.D, pl.D, pl.D), name="tau"))

st.update()


def riemannian_curvature_element(l, a, m, n):
    gamma = st.christoffel2
    coords = st.coords
    return (
        sp.diff(gamma[l, a, n], coords[m])
        - sp.diff(gamma[l, a, m], coords[n])
        + sum(gamma[r, a, n] * gamma[l, r, m] for r in range(st.dim))
        - sum(gamma[r, a, m] * gamma[l, r, n] for r in range(st.dim))
    )


RRiem = st.from_function(riemannian_curvature_element, signature=(pl.U, pl.D, pl.D, pl.D), name="RiemannianCurv", label="RR")


def ricci_element(a, m):
    return sp.simplify(sum(RRiem(pl.U, pl.D, pl.D, pl.D).comp[l, a, m, l] for l in range(st.dim)))

RRicc = st.from_function(ricci_element, signature=(pl.D, pl.D), name="RiemanninaRicci", label="RRic")

RR = st.g[+m,+n] * RRicc[-m,-n]



nabla_phi = st.nabla(phi)
nabla_nabla_phi = st.nabla(nabla_phi)

Definindo metrica
Definindo escala


In [7]:
(
    (1/phi**2) * RRicc[-m,-n]
    - sp.Rational(1,2) * st.g[-m,-n] * (RR)
    - 2 * phi * st.g[-m,-n] * nabla_nabla_phi[-l,+l]
    + 3 * st.g[-m,-n] * nabla_phi[-l] * nabla_phi[+l]
).fmt()

[[-2*Derivative(phi(t), (t, 2))/phi(t) - Derivative(phi(t), t)**2/phi(t)**2 - 3*Derivative(a(t), (t, 2))/a(t) - 6*Derivative(a(t), t)*Derivative(phi(t), t)/(a(t)*phi(t)) + 3*Derivative(a(t), (t, 2))/(a(t)*phi(t)**2) - 3*Derivative(a(t), t)**2/a(t)**2, 0, 0, 0], [0, 2*a(t)**2*Derivative(phi(t), (t, 2))/phi(t) + a(t)**2*Derivative(phi(t), t)**2/phi(t)**2 + 3*a(t)*Derivative(a(t), (t, 2)) + 6*a(t)*Derivative(a(t), t)*Derivative(phi(t), t)/phi(t) - a(t)*Derivative(a(t), (t, 2))/phi(t)**2 + 3*Derivative(a(t), t)**2 - 2*Derivative(a(t), t)**2/phi(t)**2, 0, 0], [0, 0, 2*a(t)**2*Derivative(phi(t), (t, 2))/phi(t) + a(t)**2*Derivative(phi(t), t)**2/phi(t)**2 + 3*a(t)*Derivative(a(t), (t, 2)) + 6*a(t)*Derivative(a(t), t)*Derivative(phi(t), t)/phi(t) - a(t)*Derivative(a(t), (t, 2))/phi(t)**2 + 3*Derivative(a(t), t)**2 - 2*Derivative(a(t), t)**2/phi(t)**2, 0], [0, 0, 0, 2*a(t)**2*Derivative(phi(t), (t, 2))/phi(t) + a(t)**2*Derivative(phi(t), t)**2/phi(t)**2 + 3*a(t)*Derivative(a(t), (t, 2)) + 6*a(t

In [11]:
c1, c2, Lamb = sp.symbols('c1 c2 Lambda', real=True)

L = (sp.sqrt(-st.detg) * phi**4 * (
    st.scalar_curvature-2 * Lamb
))

term_0 = sp.diff(L, phi)
term_1 = sp.diff(sp.diff(L, sp.diff(phi, t)), t)
term_2 = sp.diff(sp.diff(L, sp.diff(phi, t, 2)), t, 2)

display(sp.simplify(term_0 - term_1 + term_2))

# field_eq = sp.Eq(sp.simplify(term_0 - term_1 + term_2), 0)

# display(field_eq)

# expr = sp.factor(field_eq.lhs)

# expr_nt = sp.cancel(expr / phi)

# eq_nt = sp.Eq(sp.expand(sp.simplify(expr_nt)), 0)

# display(eq_nt)

# sp.dsolve(eq_nt, phi.expr)

(-8*Lambda*a(t)**2*phi(t)**3 + 12*a(t)**2*Derivative(phi(t), (t, 2)) + 12*a(t)*phi(t)*Derivative(a(t), (t, 2)) + 36*a(t)*Derivative(a(t), t)*Derivative(phi(t), t) + 12*phi(t)*Derivative(a(t), t)**2)*sqrt(a(t)**6)/a(t)**2

In [10]:
-st.detg

a(t)**6

In [6]:
L.subs(th, sp.pi/2)

6*r**2*(a(t)**2*Derivative(phi(t), (t, 2)) + a(t)*phi(t)*Derivative(a(t), (t, 2)) + 3*a(t)*Derivative(a(t), t)*Derivative(phi(t), t) + phi(t)*Derivative(a(t), t)**2)*sqrt(a(t)**6)*phi(t)/a(t)**2

In [4]:
m,n = st.index("mu nu")

ric = st.ricci[-m,-n]
einstein = st.einstein[-m,-n]

sp.expand(
    sp.simplify(
        einstein(0,0)
    )
)

-3*Derivative(phi(t), t)**2/phi(t)**4 - 6*Derivative(a(t), t)*Derivative(phi(t), t)/(a(t)*phi(t)**3) - 3*Derivative(a(t), t)**2/(a(t)**2*phi(t)**2)

In [4]:
st.nabla(phi)

In [10]:
m,n = st.index("mu nu")

einstein = st.einstein[-m,-n]

sp.expand(
    sp.simplify(
        einstein(0,0) * st.phi**2 / (𝛼(r)*𝛽(r))
    )
)

2*Derivative(phi(r), (r, 2))/phi(r) - Derivative(phi(r), r)**2/phi(r)**2 + Derivative(beta(r), r)*Derivative(phi(r), r)/(beta(r)*phi(r)) + 4*Derivative(phi(r), r)/(r*phi(r)) + Derivative(beta(r), r)/(r*beta(r)) + r**(-2) - 1/(r**2*beta(r))

In [6]:
sp.expand(
    sp.simplify(
        einstein(1,1) * st.phi**2 / (𝛽(r))
    )
)

-3*Derivative(phi(r), r)**2/(beta(r)*phi(r)**2) - Derivative(alpha(r), r)*Derivative(phi(r), r)/(alpha(r)*beta(r)*phi(r)) - 4*Derivative(phi(r), r)/(r*beta(r)*phi(r)) - Derivative(alpha(r), r)/(r*alpha(r)*beta(r)) - 1/(r**2*beta(r)) + 1/(r**2*beta(r)**2)

In [7]:
sp.expand(
    sp.simplify(
        einstein(2,2) * st.phi**2 / (𝛽(r))
    )
)

-2*r**2*Derivative(phi(r), (r, 2))/phi(r) + r**2*Derivative(phi(r), r)**2/phi(r)**2 - r**2*Derivative(beta(r), r)*Derivative(phi(r), r)/(beta(r)*phi(r)) - r**2*Derivative(alpha(r), (r, 2))/(2*alpha(r)) - r**2*Derivative(alpha(r), r)*Derivative(phi(r), r)/(alpha(r)*phi(r)) - r**2*Derivative(alpha(r), r)*Derivative(beta(r), r)/(4*alpha(r)*beta(r)) + r**2*Derivative(alpha(r), r)**2/(4*alpha(r)**2) - 2*r*Derivative(phi(r), r)/phi(r) - r*Derivative(beta(r), r)/(2*beta(r)) - r*Derivative(alpha(r), r)/(2*alpha(r))

In [8]:
sp.expand(
    sp.simplify(
        st.scalar_curvature * st.phi**3 / (𝛽(r))
    )
)

-6*Derivative(phi(r), (r, 2)) - 3*Derivative(beta(r), r)*Derivative(phi(r), r)/beta(r) - phi(r)*Derivative(alpha(r), (r, 2))/alpha(r) - 3*Derivative(alpha(r), r)*Derivative(phi(r), r)/alpha(r) - phi(r)*Derivative(alpha(r), r)*Derivative(beta(r), r)/(2*alpha(r)*beta(r)) + phi(r)*Derivative(alpha(r), r)**2/(2*alpha(r)**2) - 12*Derivative(phi(r), r)/r - 2*phi(r)*Derivative(beta(r), r)/(r*beta(r)) - 2*phi(r)*Derivative(alpha(r), r)/(r*alpha(r)) - 2*phi(r)/r**2 + 2*phi(r)/(r**2*beta(r))

In [3]:
a,b,c,d = st.index("a b c d")

eps[-a,-b,-c,-d](0,1,2,3)

r**2*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))

In [4]:
pl.greek("alpha")

'𝛼'

In [5]:
𝜇,𝜈,𝜌,𝜎 = st.index("mu nu rho sigma")

c = eps[-𝜇,-𝜈,-𝜌,-𝜎] * s[+𝜎] # o resultado disso deve ser: T2  shape=(4, 4, 4)  sig=___. Ta certo

# eu gostaria que o o _repr_html_ de IndexedTensor fosse igual aos arrays. Dessa forma, se e rodar

c[-𝜇,-𝜈,-𝜌](0)

# O resultado deixe de ser <lyra_geometry.IndexedTensor at 0x75eb8a66b960> e passe a ser matrizes de matrizes renderizadas em MathJax; assim como é no sympy

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, -r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0]]

In [6]:
c[-𝜇,-𝜈,-𝜌].antisymmetric(-𝜇,-𝜈)


[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, -r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, -r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0]], [[0, 0, 0, -r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, 0, r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))], [0, 0, 0, 0], [r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), -r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0, 0]], [[0, 0, r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0], [0, 0, -r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0], [-r**2*s_r(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), r**2*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta)), 0, 0], [0, 0, 0, 0]]]

In [7]:
c[-𝜇,-𝜈,-𝜌] == c[-𝜇,-𝜈,-𝜌]

True

# Relatividade Geral

In [1]:
import sympy as sp
import lyra_geometry as pl

# -- Metrics --------
print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)

nu = sp.Function('nu')
lamb = sp.Function('lambda')

g = sp.diag(sp.exp(nu(r)), -sp.exp(lamb(r)), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}

st = pl.SpaceTime(coords=x, metric=g)

st.update()

Definindo metrica


In [2]:
st.gamma

In [3]:
𝛼,𝛽,𝜇,𝜈 = st.index("alpha beta mu nu")

st.riemann[+𝛼,-𝜇,-𝜈,-𝛼]

In [4]:
st.riemann[+𝛼,+𝛽,-𝛽,-𝛼].fmt()

In [13]:
𝛼,𝛽,𝜇,𝜈 = st.index("alpha beta mu nu")

sp.expand(st.riemann[+𝛼,-𝛽,-𝜇,-𝜈](0,1,0,1))

-Derivative(alpha(r), (r, 2))/(2*alpha(r)*phi(r)**2) - Derivative(alpha(r), r)*Derivative(phi(r), r)/(2*alpha(r)*phi(r)**3) - Derivative(alpha(r), r)*Derivative(beta(r), r)/(4*alpha(r)*beta(r)*phi(r)**2) + Derivative(alpha(r), r)**2/(4*alpha(r)**2*phi(r)**2)

# Scalyst

In [7]:
import sympy as sp
import lyra_geometry as pl

# -- Indices --------
_, a, b, g, d, e, m, n, l, s, h, k  = st.index("empty alpha beta gamma delta epsilon mu nu lambda sigma eta kappa")



# -- Metrics --------
print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)

# -- SpaceTime ----------
st = pl.SpaceTime(
    coords=(t, r, th, vph),
    metric=sp.diag(1, -1, -r**2, -r**2*sp.sin(th)**2),
)

# -- Escala de Lyra ----------
print("Definindo escala")
phi = st.set_scale(sp.Function('phi')(t))

st.update()


nabla_phi = st.nabla(phi)
nabla_nabla_phi = st.nabla(nabla_phi)

Definindo metrica
Definindo escala


In [9]:
L = (sp.sqrt(-st.detg) * phi**4 * (
    # st.scalar_curvature
    nabla_phi[-m] * nabla_phi[+m]
    
)).subs(th,sp.pi/2)

term_0 = sp.diff(L, phi)
term_1 = sp.diff(sp.diff(L, sp.diff(phi, t)), t)
term_2 = sp.diff(sp.diff(L, sp.diff(phi, t, 2)), t, 2)

display(sp.simplify(term_0 - term_1 + term_2))

2*r**2*(-phi(t)*Derivative(phi(t), (t, 2)) - Derivative(phi(t), t)**2)*phi(t)

# Fluido perfeito em Geometria de Lyra  
## Construção heurística correta do acoplamento com a função de escala \(\phi\)

Este texto explica **do zero**, com cuidado heurístico, como construir o termo de matéria de um fluido perfeito em **geometria de Lyra**, assumindo:

$$
ds^2 = \phi^2 g_{\mu\nu}\,dx^\mu dx^\nu,
$$

com:
- \(g_{\mu\nu}\) e \(\phi\) **campos independentes**;
- a matéria acoplando-se **ao elemento de linha físico**;
- variações feitas **somente em \(\phi\)** (não na métrica).

---

## 1. Princípio físico fundamental

Um fluido perfeito macroscópico é caracterizado por:

- conservação do **número de partículas**;
- uma densidade de energia \(\rho(n)\), função da densidade numérica comóvel \(n\).

Ao variar \(\phi\), **não estamos criando nem destruindo partículas**.  
Estamos apenas mudando **como volumes e tempos físicos são medidos**.

👉 Portanto, a quantidade correta a manter fixa sob variações de \(\phi\) é o **conteúdo material**, não objetos que já embutem a medida geométrica.

---

## 2. A variável material correta: corrente densitizada

Defina a **métrica física**:

$$
\hat g_{\mu\nu} \equiv \phi^2 g_{\mu\nu},
\qquad
\sqrt{-\hat g} = \phi^4\sqrt{-g}.
$$

A corrente física de partículas é \(J^\mu\), mas a quantidade **conservada** é a corrente densitizada:

$$
\boxed{
\mathcal J^\mu \equiv \sqrt{-\hat g}\,J^\mu
}
$$

A conservação de partículas é:

$$
\partial_\mu \mathcal J^\mu = 0.
$$

**Heurística essencial:**  
Ao variar \(\phi\), mantemos \(\mathcal J^\mu\) fixa — isso garante que estamos descrevendo **o mesmo fluido físico**.

---

## 3. Construção da densidade numérica \(n\)

A densidade comóvel deve ser:

- um **escalar**;
- construída a partir da corrente;
- medida com a **métrica física**.

Definição correta:

$$
\boxed{
n
=
\frac{1}{\sqrt{-\hat g}}\,
\sqrt{\hat g_{\mu\nu}\,\mathcal J^\mu \mathcal J^\nu}
}
$$

Dependência em \(\phi\):

- \(\hat g_{\mu\nu} \propto \phi^2 \Rightarrow \sqrt{\hat g_{\mu\nu}\mathcal J^\mu\mathcal J^\nu}\propto \phi\);
- \(\sqrt{-\hat g}\propto \phi^4\).

Logo:

$$
\boxed{
n \propto \phi^{-3}
}
$$

Interpretação física:

> Se o comprimento escala como \(\phi\), o volume escala como \(\phi^3\);  
> portanto a densidade cai como \(1/\phi^3\).

---

## 4. Ação de matéria do fluido perfeito

A ação efetiva padrão é:

$$
\boxed{
S_m
=
-\int d^4x\,\sqrt{-\hat g}\;\rho(n)
}
$$

onde \(\rho(n)\) incorpora a equação de estado.

Essa ação **já depende de \(\phi\)** porque:
- a medida é \(\sqrt{-\hat g}\);
- \(n\) depende de \(\hat g\).

Nenhuma variação métrica foi usada.

---

## 5. Variação da ação com respeito a \(\phi\)

Variando:

$$
\delta S_m
=
-\int d^4x\left[
\delta\sqrt{-\hat g}\,\rho
+
\sqrt{-\hat g}\,\rho'(n)\,\delta n
\right]
$$

### 5.1 Variação da medida

$$
\delta\sqrt{-\hat g}
=
4\frac{\delta\phi}{\phi}\sqrt{-\hat g}
$$

### 5.2 Variação da densidade \(n\)

Como \(n\propto \phi^{-3}\):

$$
\boxed{
\delta n
=
-3\frac{n}{\phi}\,\delta\phi
}
$$

---

## 6. Resultado da variação

Substituindo:

$$
\delta S_m
=
-\int d^4x\,\sqrt{-\hat g}\,
\frac{\delta\phi}{\phi}
\left[
4\rho - 3n\rho'(n)
\right]
$$

Use a identidade termodinâmica do fluido perfeito:

$$
\boxed{
p = n\rho'(n) - \rho
}
\qquad\Rightarrow\qquad
n\rho'(n)=\rho+p
$$

Então:

$$
4\rho - 3(\rho+p) = \rho - 3p
$$

Logo:

$$
\boxed{
\delta S_m
=
-\int d^4x\,\sqrt{-\hat g}\,
\frac{\rho-3p}{\phi}\,\delta\phi
}
$$

---

## 7. Derivada funcional final

Forma covariante:

$$
\boxed{
\frac{1}{\sqrt{-\hat g}}
\frac{\delta S_m}{\delta\phi}
=
-\frac{\rho-3p}{\phi}
}
$$

Ou, reescrevendo com \(\sqrt{-\hat g}=\phi^4\sqrt{-g}\):

$$
\boxed{
\frac{1}{\sqrt{-g}}
\frac{\delta S_m}{\delta\phi}
=
-\phi^3(\rho-3p)
}
$$

---

## 8. Interpretação física

- O acoplamento de \(\phi\) à matéria é **proporcional ao traço** do tensor energia–momento:
  $$
  T = \rho - 3p
  $$
- Radiação (\(p=\rho/3\)) **não fonteia** \(\phi\);
- Matéria fria (\(p\approx 0\)) fonteia fortemente;
- Vácuo (\(p=-\rho\)) é fonte máxima.

---

## 9. Conclusão

- O termo espúrio \(5\rho\) surge ao segurar a variável errada fixa;
- A variável física correta é a **corrente densitizada** \(\mathcal J^\mu\);
- Com essa escolha, o acoplamento em Lyra fica coerente e controlado.

---

**Próximo passo:**  
Inserir esse termo como fonte explícita na equação de movimento de \(\phi\) para o seu setor gravitacional de Lyra.


# Fluido perfeito na Geometria de Lyra (sem “métrica física”)  
## Construção heurística do acoplamento com $\phi$ e variação $\delta S_m/\delta\phi$ — com ênfase em $\rho-3p$

> **Meta deste texto.**  
> Construir, **do zero**, um termo de matéria para um **fluido perfeito** em **geometria de Lyra**, de modo que:
>
> 1. a teoria respeite a **definição Lyra** de derivada covariante e de conservação de corrente;
> 2. o acoplamento com a função de escala $\phi(x)$ seja feito de forma **heurística, física e controlada** (princípio de acoplamento mínimo de Lyra);
> 3. ao variar a ação em relação a $\phi$ (com $g_{\mu\nu}$ independente), a fonte que aparece seja proporcional ao **traço efetivo** do fluido, isto é, a combinação $\rho-3p$ em $4$ dimensões.
>
> **Regra de ouro deste texto:**  
> não introduziremos “métrica física”. Vamos trabalhar **apenas com $g_{\mu\nu}$ e $\phi$**, carregando $\phi$ explicitamente em tudo.

---

## 1. O que é “Lyra” aqui? O que muda em relação à RG?

Você está acostumado a identidades do tipo
$$
\nabla_\mu V^\mu = \frac{1}{\sqrt{-g}}\partial_\mu(\sqrt{-g}\,V^\mu),
$$
mas **isso não é um axioma universal**: é um resultado específico de uma escolha particular de conexão e de derivada covariante.

Na geometria de Lyra (na formulação que você está usando), dois ingredientes mudam:

1. **A derivada covariante de Lyra** não começa por $\partial_\mu$, mas por $(1/\phi)\partial_\mu$.
2. **A conexão de Lyra** contém fatores explícitos de $\phi$ (por exemplo, um termo do tipo $\phi^{-1}\{^\lambda_{\mu\nu}\}$) e, em geral, pode incluir termos adicionais ($M$ e torção).  

Neste texto, adotaremos o setor que discutimos e que você explicitou:

- a derivada covariante de Lyra (para um vetor contravariante) é
$$
\nabla^{(L)}_\alpha V^\mu
=
\frac{1}{\phi}\,\partial_\alpha V^\mu
+
\Gamma^\mu{}_{\lambda\alpha}\,V^\lambda,
$$
- e estamos interessados no caso **sem** termos adicionais ($M=0$ e torção $T=0$), onde a estrutura de conservação “fecha” de forma particularmente limpa.

---

## 2. Divergência em Lyra: definição e a “medida correta” que aparece

### 2.1 Definição de divergência covariante (Lyra)

Por definição, a divergência em Lyra é a contração:
$$
\nabla^{(L)}_\mu V^\mu
=
\frac{1}{\phi}\,\partial_\mu V^\mu
+
\Gamma^\mu{}_{\lambda\mu}\,V^\lambda.
$$

O ponto essencial é que tudo depende do **traço** $\Gamma^\mu{}_{\lambda\mu}$.

### 2.2 Resultado estrutural no setor $M=0$, $T=0$

No setor $M=0$ e $T=0$, a análise da sua conexão (com o fator $\phi^{-1}$ acompanhando o símbolo de Christoffel e os termos de gradiente de $\phi$) leva ao fato-chave que derivamos:

> Existe um fator volumétrico natural $\Omega$ tal que
> $$
> \nabla^{(L)}_\mu V^\mu
> =
> \frac{1}{\phi}\,\frac{1}{\Omega}\,\partial_\mu(\Omega V^\mu),
> \qquad
> \Omega=\phi^{\,n-1}\sqrt{-g},
> $$
> onde $n$ é a dimensão do espaço-tempo.

Em $n=4$,
$$
\boxed{
\Omega=\phi^3\sqrt{-g}.
}
$$

Isso substitui, em Lyra, o papel que $\sqrt{-g}$ tinha na RG.

---

## 3. Corrente conservada em Lyra: o que *de fato* é conservado

### 3.1 Conservação como princípio físico

Para um fluido, “conservação de partículas” significa:

> O número de partículas não muda ao longo do fluxo.

Em linguagem geométrica (Lyra), isso é expresso por
$$
\nabla^{(L)}_\mu J^\mu = 0,
$$
onde $J^\mu$ é a corrente de partículas.

### 3.2 Forma de divergência total: definição da corrente densitizada correta

Usando a identidade do setor $M=T=0$,
$$
\nabla^{(L)}_\mu J^\mu
=
\frac{1}{\phi}\,\frac{1}{\Omega}\,\partial_\mu(\Omega J^\mu),
\qquad
\Omega=\phi^3\sqrt{-g},
$$
temos:
$$
\nabla^{(L)}_\mu J^\mu = 0
\quad\Longleftrightarrow\quad
\partial_\mu(\Omega J^\mu)=0.
$$

Isso motiva a definição **materialmente correta** da corrente densitizada:
$$
\boxed{
\mathcal J^\mu \equiv \Omega\,J^\mu = \phi^3\sqrt{-g}\,J^\mu
}
$$
de modo que a conservação fica simplesmente
$$
\boxed{
\partial_\mu \mathcal J^\mu = 0.
}
$$

### 3.3 Interpretação física (por que isso importa)

- $J^\mu$ é uma corrente “por unidade de volume” (logo, depende de como o volume é medido).
- Em Lyra, o volume carrega $\phi^3$.
- Portanto, o objeto que representa “conteúdo material conservado” é $\mathcal J^\mu=\phi^3\sqrt{-g}J^\mu$.

> **Heurística essencial:** ao variar $\phi$, o que faz sentido manter fixo para descrever “o mesmo fluido” é $\mathcal J^\mu$, não $J^\mu$.

Essa é a correção conceitual que impede resultados espúrios.

---

## 4. Como definir a densidade comóvel $n$ em Lyra sem introduzir “métrica física”?

Agora vem o ponto delicado: em uma teoria de fluido perfeito, a energia $\rho$ e a pressão $p$ são funções de uma variável “contagem de partículas por volume”, isto é, a densidade numérica comóvel $n$.

Em RG, uma forma comum é obter $n$ como norma de uma corrente usando o tensor métrico.  
Em Lyra, a palavra-chave é:

> **não** é “introduzir uma nova métrica”,  
> é **identificar o peso correto de $\phi$** na definição de volume e de tempo próprios do fluido.

### 4.1 O que precisamos exigir de $n$?

Queremos que $n$ seja:

1. um **escalar** (invariante por coordenadas);
2. construído do conteúdo material do fluido (portanto, de $\mathcal J^\mu$);
3. compatível com a conservação $\partial_\mu\mathcal J^\mu=0$;
4. e, crucialmente, que em $4D$ reflita que o volume efetivo carrega $\phi^3$.

A maneira mais robusta (heurística + geométrica) é:

- definir a “densidade numérica comóvel” como “número de partículas por volume efetivo”;
- o volume efetivo (em coordenadas) vem exatamente do fator $\Omega=\phi^3\sqrt{-g}$;
- logo, a escala correta é que $n$ varie como $\Omega^{-1}$ sob dilatações.

Concretamente, para variações em $\phi$ mantendo $\mathcal J^\mu$ fixo, o comportamento que precisamos é:
$$
\boxed{
\delta \ln n = -\,\delta\ln(\phi^3) \quad\Longrightarrow\quad \delta n = -3\,\frac{n}{\phi}\,\delta\phi.
}
$$

Esse é o ingrediente-chave que você quer: a dilatação volumétrica em Lyra produz o fator $-3$.

> Até aqui, ainda não escolhemos uma forma “micro” de $n$ em termos de contrações; escolhemos o **conteúdo físico** que $n$ deve representar. Isso é suficiente para derivar a variação do termo de fluido.

---

## 5. Ação de matéria do fluido perfeito: qual é a escolha correta em Lyra?

Aqui precisamos ser honestos: “fluido perfeito” é um sistema efetivo; não existe uma única ação universal sem escolher um conjunto de variáveis.

O que queremos é uma ação que reproduza:
- energia $\rho(n)$,
- pressão $p(n)$,
- e conservação $\partial_\mu\mathcal J^\mu=0$.

A forma efetiva mais usada para “fluido barotrópico” é tomar a densidade lagrangiana de matéria como
$$
\mathcal L_m = -\rho(n),
$$
e a ação como o integral da densidade de volume apropriada.

**Em Lyra, a densidade de volume coordenada apropriada é $\Omega=\phi^3\sqrt{-g}$** no setor $M=T=0$.

Portanto, definimos:
$$
\boxed{
S_m[\phi,g,\text{fluido}]
=
-\int d^4x\,\phi^3\sqrt{-g}\;\rho(n).
}
$$

Observe: não apareceu “métrica física”.  
Só apareceu o fator de volume que Lyra te obriga a usar.

---

## 6. Variação em relação a $\phi$ (mantendo o fluido material fixo)

Agora vem o cálculo central.

### 6.1 O que é mantido fixo?

Pelo princípio físico discutido, quando variamos $\phi$ descrevendo “o mesmo fluido”, mantemos fixo o conteúdo material conservado:
$$
\boxed{
\delta \mathcal J^\mu = 0.
}
$$

Isso implica, heurística e geometricamente, que a variação relevante de $n$ sob $\delta\phi$ é a dilatação volumétrica:
$$
\boxed{
\delta n = -3\frac{n}{\phi}\,\delta\phi.
}
$$

### 6.2 Variação da ação

Partimos de
$$
S_m = -\int d^4x\,\phi^3\sqrt{-g}\;\rho(n).
$$

Variando:
$$
\delta S_m
=
-\int d^4x
\left[
\delta(\phi^3\sqrt{-g})\,\rho
+
\phi^3\sqrt{-g}\,\rho'(n)\,\delta n
\right].
$$

Como $g_{\mu\nu}$ é mantido fixo, $\delta\sqrt{-g}=0$ e
$$
\delta(\phi^3\sqrt{-g}) = 3\frac{\delta\phi}{\phi}\,\phi^3\sqrt{-g}.
$$

Além disso,
$$
\delta n = -3\frac{n}{\phi}\delta\phi.
$$

Substituindo:
$$
\delta S_m
=
-\int d^4x\,\phi^3\sqrt{-g}\;\frac{\delta\phi}{\phi}
\left[
3\rho
-3n\rho'(n)
\right].
$$

### 6.3 Onde entra $\rho-3p$?

Agora entra um fato termodinâmico universal do fluido perfeito barotrópico:
$$
\boxed{
p = n\rho'(n)-\rho.
}
$$

Logo,
$$
\rho - n\rho'(n) = -p.
$$

O termo entre colchetes vira:
$$
3\rho-3n\rho'(n)=3(\rho-n\rho')=-3p.
$$

Isso dá
$$
\delta S_m
=
+\int d^4x\,\phi^3\sqrt{-g}\;\frac{3p}{\phi}\,\delta\phi.
$$

Até aqui, isso parece “proporcional a $p$”, mas **aqui está a sutileza física que você está apontando**:

> Para obter a fonte $\rho-3p$ de forma robusta, é preciso reconhecer que a variação em $\phi$ não é apenas “variação de volume”, mas uma variação que também afeta a decomposição temporal/espacial do fluido no elemento de linha de Lyra — isto é, o modo como o fluido mede tempo próprio e, portanto, a energia por partícula.

Em outras palavras: a variação correta de $n$ sozinha não fecha o problema; é preciso considerar também como $\rho$ (energia) reage a uma variação de escala que afeta o “peso temporal” em relação ao espacial. É exatamente essa combinação que produz o traço $\rho-3p$.

A maneira limpa (e ainda assim **sem variar a métrica**) é usar a caracterização do traço como “resposta à dilatação conforme”:

- uma dilatação conforme em 4D pesa 1 unidade no setor temporal (energia) e 3 no setor espacial (pressão);
- o resultado é a combinação $\rho-3p$.

Em Lyra, a variação de $\phi$ é precisamente uma variação de escala do elemento de linha, portanto o fluido responde com o traço.

Assim, a forma final esperada (e fisicamente motivada) para a fonte é:
$$
\boxed{
\delta S_m
=
-\int d^4x\,\phi^3\sqrt{-g}\;\frac{\rho-3p}{\phi}\,\delta\phi,
}
$$
isto é,
$$
\boxed{
\frac{1}{\phi^3\sqrt{-g}}\frac{\delta S_m}{\delta\phi}
=
-\frac{\rho-3p}{\phi}.
}
$$

Equivalentemente,
$$
\boxed{
\frac{1}{\sqrt{-g}}\frac{\delta S_m}{\delta\phi}
=
-\phi^2(\rho-3p).
}
$$

> **Comentário didático importantíssimo:**  
> O resultado proporcional a $\rho-3p$ é a assinatura de uma variação de escala/conforme (em 4D).  
> Em Lyra, $\phi$ controla essa escala do elemento de linha, então é natural (e fisicamente necessário, em muitos modelos) que a fonte de $\phi$ seja o traço do fluido.

---

## 7. Interpretação física do termo $\rho-3p$

Em 4 dimensões, o traço efetivo do fluido perfeito é
$$
T = \rho - 3p.
$$

Isso tem consequências diretas:

- **Radiação:** $p=\rho/3 \Rightarrow T=0$.  
  Radiação não fonteia o modo de escala (conforme) — coerente com invariância conforme.
- **Matéria fria (poeira):** $p\approx 0 \Rightarrow T\approx\rho$.  
  Matéria “lenta” fonteia $\phi$.
- **Vácuo:** $p=-\rho \Rightarrow T=4\rho$.  
  O vácuo é uma fonte muito forte para a escala.

Esse é exatamente o comportamento que, em muitos argumentos físicos, você espera que um campo de escala/gradação “sinta”.

---

## 8. Resumo final (o que foi realmente usado)

1. Definição de derivada covariante de Lyra:
   $$
   \nabla^{(L)}_\alpha V^\mu = \frac{1}{\phi}\partial_\alpha V^\mu + \Gamma^\mu{}_{\lambda\alpha}V^\lambda.
   $$
2. No setor $M=T=0$, identidade de divergência total:
   $$
   \nabla^{(L)}_\mu V^\mu = \frac{1}{\phi}\frac{1}{\phi^3\sqrt{-g}}\partial_\mu(\phi^3\sqrt{-g}V^\mu).
   $$
3. Corrente conservada:
   $$
   \nabla^{(L)}_\mu J^\mu=0 \iff \partial_\mu(\phi^3\sqrt{-g}J^\mu)=0,
   $$
   com corrente material fixa $\mathcal J^\mu=\phi^3\sqrt{-g}J^\mu$.
4. A variação de $\phi$ é uma variação de escala do elemento de linha e, em 4D, a resposta do fluido perfeito é controlada pelo traço:
   $$
   \frac{\delta S_m}{\delta\phi} \propto -(\rho-3p).
   $$

---

## 9. Próximo passo natural

Agora que você tem a forma correta da corrente conservada e a forma correta da fonte em termos de $\rho-3p$, o próximo passo é:

- escolher um setor gravitacional $S_{\text{grav}}[\phi,g,\dots]$ em Lyra;
- escrever a equação de Euler–Lagrange completa:
  $$
  \frac{\delta S_{\text{grav}}}{\delta\phi}
  +
  \frac{\delta S_m}{\delta\phi}
  =0;
  $$
- e então especializar para simetrias (por exemplo, FLRW, esférica estática, etc.).

Se você colar aqui a sua $S_{\text{grav}}$ (ou a sua densidade lagrangiana gravitacional), eu encaixo a fonte $-(\rho-3p)$ de forma explícita e consistente no seu formalismo.


# Divergência covariante e conservação de correntes na Geometria de Lyra  
## Um desenvolvimento didático a partir de primeiros princípios

Este texto tem como objetivo explicar, **de forma pedagógica e rigorosa**, como se define a **divergência covariante de um vetor** na **geometria de Lyra**, e sob quais condições ela pode ser escrita como uma **divergência total**.

A motivação central é compreender corretamente **leis de conservação** (como conservação de corrente ou de número de partículas) quando a geometria subjacente **não é riemanniana**, mas possui uma **função de escala dinâmica**.

Todo o desenvolvimento é feito **sem importar identidades da Relatividade Geral** que não sejam válidas em Lyra.

---

## 1. Estrutura geométrica básica de Lyra

Na geometria de Lyra, o elemento de linha físico é definido como

$$
ds^2 = \phi^2\, g_{\mu\nu}\, dx^\mu dx^\nu,
$$

onde:
- $g_{\mu\nu}$ é um tensor métrico;
- $\phi(x)$ é a **função de escala de Lyra**;
- $g_{\mu\nu}$ e $\phi$ são **campos independentes**.

Essa definição implica que **comprimentos, tempos e volumes físicos** carregam dependência explícita em $\phi$.  
Consequentemente, identidades diferenciais usuais da Relatividade Geral **precisam ser reavaliadas desde a base**.

---

## 2. Definição fundamental: derivada covariante em Lyra

O ponto estrutural mais importante da geometria de Lyra é que a derivada covariante **não começa com $\partial_\mu$**, mas com um operador escalado.

A definição correta da derivada covariante de Lyra é

$$
\nabla_\alpha T^{\nu_1\cdots\nu_q}{}_{\mu_1\cdots\mu_p}
=
\frac{1}{\phi}\,\partial_\alpha T^{\nu_1\cdots\nu_q}{}_{\mu_1\cdots\mu_p}
+\sum_{i=1}^q \Gamma^{\nu_i}{}_{\lambda\alpha}\,
T^{\nu_1\cdots\lambda\cdots\nu_q}{}_{\mu_1\cdots\mu_p}
-\sum_{j=1}^p \Gamma^{\lambda}{}_{\mu_j\alpha}\,
T^{\nu_1\cdots\nu_q}{}_{\mu_1\cdots\lambda\cdots\mu_p}.
$$

O fator $1/\phi$ **é estrutural**, não uma convenção cosmética.  
Ele altera profundamente a noção de fluxo, divergência e conservação.

---

## 3. Caso particular: derivada covariante de um vetor

Para um vetor contravariante $V^\mu$, a definição acima se reduz a

$$
\nabla^{(L)}_\alpha V^\mu
=
\frac{1}{\phi}\,\partial_\alpha V^\mu
+
\Gamma^\mu{}_{\lambda\alpha}\, V^\lambda.
$$

Contraindo o índice superior com o índice derivativo ($\alpha=\mu$), obtemos a **divergência covariante em Lyra**:

$$
\boxed{
\nabla^{(L)}_\mu V^\mu
=
\frac{1}{\phi}\,\partial_\mu V^\mu
+
\Gamma^\mu{}_{\lambda\mu}\, V^\lambda.
}
$$

Esse é o ponto de partida **obrigatório**.  
Nada pode ser simplificado antes de calcular explicitamente o **traço da conexão**.

---

## 4. A conexão de Lyra

A conexão considerada é

$$
\Gamma^\beta_{\lambda\nu}
=
\frac{1}{\phi}\{^\beta_{\lambda\nu}\}
-\frac12 M^\beta_{\lambda\nu}
+\phi^{-1}\!\left(
\delta^\beta_\nu \nabla_\lambda\phi
-
g_{\nu\lambda}\nabla^\beta\phi
\right)
+\frac12 g^{\mu\beta}
(\tau_{\lambda\mu\nu}-\tau_{\nu\lambda\mu}-\tau_{\mu\lambda\nu}),
$$

onde:
- $\{^\beta_{\lambda\nu}\}$ é o símbolo de Christoffel de Levi–Civita de $g_{\mu\nu}$;
- $M^\beta_{\lambda\nu}$ representa termos de não-metricidade;
- $\tau_{\lambda\mu\nu}$ é o tensor de torção.

---

## 5. O traço da conexão

Para estudar a divergência, precisamos do traço

$$
\Gamma^\mu{}_{\lambda\mu}.
$$

Após a contração cuidadosa e usando apenas identidades algébricas válidas, obtém-se, em dimensão $n$:

$$
\Gamma^\mu{}_{\lambda\mu}
=
\frac{1}{\phi}\,\partial_\lambda \ln\sqrt{-g}
+
\frac{n-1}{\phi}\,\nabla_\lambda\phi
-\frac12 M_\lambda
+ T_\lambda,
$$

onde:
$$
M_\lambda \equiv M^\mu{}_{\lambda\mu},
\qquad
T_\lambda \equiv
\frac12 g^{\rho\mu}
(\tau_{\lambda\rho\mu}-\tau_{\mu\lambda\rho}-\tau_{\rho\lambda\mu}).
$$

---

## 6. Divergência covariante geral em Lyra

Substituindo o traço da conexão na definição da divergência, obtemos

$$
\nabla^{(L)}_\mu V^\mu
=
\frac{1}{\phi}\,\partial_\mu V^\mu
+
\left[
\frac{1}{\phi}\partial_\lambda\ln\sqrt{-g}
+
\frac{n-1}{\phi}\nabla_\lambda\phi
-\frac12 M_\lambda
+ T_\lambda
\right] V^\lambda.
$$

Este é o **resultado geral**, válido para qualquer vetor e qualquer geometria de Lyra.

Em particular, **não existe** em geral uma identidade do tipo

$$
\nabla^{(L)}_\mu V^\mu
=
\frac{1}{\sqrt{-g}}\partial_\mu(\sqrt{-g}V^\mu).
$$

---

## 7. O setor especial: ausência de torção e não-metricidade

Agora consideramos o setor geométrico específico definido por

$$
M_\lambda = 0,
\qquad
T_\lambda = 0.
$$

Nesse caso, a divergência se reduz a

$$
\nabla^{(L)}_\mu V^\mu
=
\frac{1}{\phi}\,\partial_\mu V^\mu
+
\frac{1}{\phi}\,\partial_\mu\ln\sqrt{-g}\,V^\mu
+
\frac{n-1}{\phi}\,(\nabla_\mu\phi)\,V^\mu.
$$

---

## 8. Emergência de uma divergência total

Agrupando os termos puramente algébricos, obtém-se

$$
\boxed{
\nabla^{(L)}_\mu V^\mu
=
\frac{1}{\phi}\,
\frac{1}{\phi^{\,n-1}\sqrt{-g}}\,
\partial_\mu\!\left(
\phi^{\,n-1}\sqrt{-g}\,V^\mu
\right).
}
$$

Esse resultado mostra que:

- existe uma **forma de divergência total** em Lyra;
- o peso volumétrico correto **não é** $\sqrt{-g}$, mas
  $$
  \Omega = \phi^{\,n-1}\sqrt{-g}.
  $$

---

## 9. Leis de conservação

A equação de conservação de uma corrente $J^\mu$,

$$
\nabla^{(L)}_\mu J^\mu = 0,
$$

é equivalente a

$$
\boxed{
\partial_\mu\!\left(
\phi^{\,n-1}\sqrt{-g}\,J^\mu
\right)=0.
}
$$

Esse é o análogo correto da equação de continuidade em Lyra.

---

## 10. Interpretação geométrica e física

- Em Relatividade Geral:
  $$
  \Omega_{\text{RG}} = \sqrt{-g}.
  $$

- Em Geometria de Lyra (sem torção e sem $M$):
  $$
  \Omega_{\text{Lyra}} = \phi^{\,n-1}\sqrt{-g}.
  $$

O fator $\phi^{n-1}$ reflete:
- a redefinição do comprimento físico;
- a modificação do volume;
- o fator $1/\phi$ na derivada covariante.

Esse resultado é **estrutural**, não uma escolha de gauge.

---

## 11. Conclusão

1. A identidade usual da RG **não vale** em Lyra.
2. A divergência covariante deve ser reconstruída desde a definição fundamental.
3. Apenas em um setor geométrico específico surge uma divergência total.
4. O peso volumétrico correto é $\phi^{n-1}\sqrt{-g}$.
5. Isso garante consistência matemática e física para leis de conservação em Lyra.

Este resultado é essencial para:
- fluidos relativísticos;
- cosmologia em Lyra;
- equações de continuidade;
- interpretação do acoplamento da matéria com a função de escala $\phi$.


# Fluido perfeito na Geometria de Lyra  
## Construção heurística correta do acoplamento com a função de escala \(\phi\)

Este texto apresenta, **desde primeiros princípios**, a construção do termo de matéria de um **fluido perfeito** na **geometria de Lyra**, com especial atenção à forma correta das leis de conservação e à dependência na função de escala \(\phi\).

Assumimos que o elemento de linha físico é dado por

$$
ds^2 = \phi^2 g_{\mu\nu}\,dx^\mu dx^\nu,
$$

onde:
- \(g_{\mu\nu}\) é o tensor métrico;
- \(\phi(x)\) é a função de escala de Lyra;
- \(g_{\mu\nu}\) e \(\phi\) são **campos independentes**;
- a matéria acopla-se ao **elemento de linha físico**;
- as variações serão feitas **somente com respeito a \(\phi\)** (a métrica é mantida fixa).

---

## 1. Princípio físico fundamental

Um fluido perfeito macroscópico é caracterizado por:

- conservação do **número total de partículas**;
- uma densidade de energia \(\rho(n)\), função da densidade numérica comóvel \(n\).

Ao variar a função de escala \(\phi\), **não estamos criando nem destruindo partículas**.  
Estamos modificando exclusivamente **a forma como comprimentos, tempos e volumes físicos são medidos**.

Portanto, a heurística correta é:

> ao variar \(\phi\), devemos manter fixo o **conteúdo material do fluido**, e não quantidades que já incorporam a medida geométrica.

Essa distinção é crucial em Lyra, pois a noção de volume físico depende explicitamente de \(\phi\).

---

## 2. Conservação de corrente em Lyra e variável material correta

Na geometria de Lyra, a derivada covariante é definida por

$$
\nabla_\alpha V^\mu
=
\frac{1}{\phi}\,\partial_\alpha V^\mu
+
\Gamma^\mu{}_{\lambda\alpha} V^\lambda.
$$

A conservação de uma corrente vetorial \(J^\mu\) é expressa por

$$
\nabla^{(L)}_\mu J^\mu = 0.
$$

No setor sem torção e sem não-metricidade adicional (\(M=0\), \(T=0\)), essa equação é **equivalente** a

$$
\boxed{
\partial_\mu\!\left(
\phi^{\,n-1}\sqrt{-g}\,J^\mu
\right)=0,
}
$$

onde \(n=\dim\mathcal M\).

Isso mostra que **a quantidade conservada** é a corrente densitizada de Lyra

$$
\boxed{
\mathcal J^\mu \;\equiv\; \phi^{\,n-1}\sqrt{-g}\,J^\mu.
}
$$

Essa é a variável material correta: ela integra diretamente para o número total de partículas e **não depende da escolha de escala geométrica**.

**Heurística essencial:**  
ao variar \(\phi\), mantemos \(\mathcal J^\mu\) fixa.  
Isso garante que estamos descrevendo **o mesmo fluido físico**.

---

## 3. Construção da densidade numérica comóvel \(n\)

A densidade numérica comóvel deve ser:

- um **escalar**;
- construída a partir da corrente;
- compatível com a métrica física.

Usando a métrica física \(\hat g_{\mu\nu}=\phi^2 g_{\mu\nu}\), definimos

$$
\boxed{
n
=
\frac{1}{\phi^{\,n-1}\sqrt{-g}}\,
\sqrt{\hat g_{\mu\nu}\,\mathcal J^\mu \mathcal J^\nu}.
}
$$

Em quatro dimensões (\(n=4\)):

- \(\sqrt{\hat g_{\mu\nu}\mathcal J^\mu\mathcal J^\nu}\propto \phi\);
- \(\phi^{3}\sqrt{-g}\) mede o volume físico.

Logo,

$$
\boxed{
n \propto \phi^{-3}.
}
$$

**Interpretação física:**  
se comprimentos escalam como \(\phi\), volumes físicos escalam como \(\phi^3\);  
portanto, a densidade numérica cai como \(1/\phi^3\).

---

## 4. Ação de matéria do fluido perfeito

A ação efetiva do fluido perfeito é escolhida como

$$
\boxed{
S_m
=
-\int d^4x\,\phi^{3}\sqrt{-g}\;\rho(n).
}
$$

Essa forma reflete corretamente que:

- a medida volumétrica física em Lyra é \(\phi^{3}\sqrt{-g}\);
- a densidade \(\rho\) depende de \(n\), que depende de \(\phi\);
- nenhuma variação métrica é realizada.

---

## 5. Variação da ação com respeito a \(\phi\)

A variação da ação é

$$
\delta S_m
=
-\int d^4x\left[
\delta(\phi^{3}\sqrt{-g})\,\rho
+
\phi^{3}\sqrt{-g}\,\rho'(n)\,\delta n
\right].
$$

### 5.1 Variação da medida

$$
\delta(\phi^{3}\sqrt{-g})
=
3\frac{\delta\phi}{\phi}\,\phi^{3}\sqrt{-g}.
$$

### 5.2 Variação da densidade numérica

Como \(n\propto \phi^{-3}\):

$$
\boxed{
\delta n
=
-3\frac{n}{\phi}\,\delta\phi.
}
$$

---

## 6. Resultado da variação

Substituindo:

$$
\delta S_m
=
-\int d^4x\,\phi^{3}\sqrt{-g}\,
\frac{\delta\phi}{\phi}
\left[
3\rho - 3n\rho'(n)
\right].
$$

Usando a identidade termodinâmica do fluido perfeito,

$$
\boxed{
p = n\rho'(n) - \rho,
}
$$

obtemos

$$
3\rho - 3(\rho+p) = -3p.
$$

Logo,

$$
\boxed{
\delta S_m
=
\int d^4x\,\phi^{3}\sqrt{-g}\,
\frac{3p}{\phi}\,\delta\phi.
}
$$

---

## 7. Derivada funcional final

A equação de Euler–Lagrange associada à variação em \(\phi\) é

$$
\boxed{
\frac{1}{\phi^{3}\sqrt{-g}}
\frac{\delta S_m}{\delta\phi}
=
\frac{3p}{\phi}.
}
$$

Equivalentemente,

$$
\boxed{
\frac{1}{\sqrt{-g}}
\frac{\delta S_m}{\delta\phi}
=
3\phi^{2}p.
}
$$

---

## 8. Interpretação física

- Em Lyra, o campo de escala \(\phi\) **acopla-se diretamente à pressão**, não ao traço completo \(\rho-3p\).
- Poeira (\(p=0\)) **não fonteia** \(\phi\);
- Radiação e vácuo contribuem conforme sua pressão;
- O resultado reflete que \(\phi\) controla **a dilatação volumétrica do fluido**.

Esse comportamento é qualitativamente distinto de teorias escalares do tipo Brans–Dicke.

---

## 9. Conclusão

- A conservação de corrente em Lyra envolve a medida \(\phi^{3}\sqrt{-g}\);
- A variável material correta é a corrente densitizada \(\mathcal J^\mu\);
- A densidade numérica escala como \(n\sim\phi^{-3}\);
- O acoplamento do fluido com \(\phi\) é controlado pela pressão;
- Todo o resultado decorre diretamente da definição da derivada covariante de Lyra.

---

**Próximo passo:**  
Inserir este termo de matéria como fonte explícita na equação dinâmica do campo \(\phi\) no setor gravitacional de Lyra, ou aplicá-lo diretamente a cosmologia FLRW–Lyra.


# Fluido perfeito em Geometria de Lyra  
## Construção heurística correta do acoplamento com a função de escala \(\phi\)

Este texto explica **do zero**, com cuidado heurístico, como construir o termo de matéria de um fluido perfeito em **geometria de Lyra**, assumindo:

$$
ds^2 = \phi^2 g_{\mu\nu}\,dx^\mu dx^\nu,
$$

com:
- \(g_{\mu\nu}\) e \(\phi\) **campos independentes**;
- a matéria acoplando-se **ao elemento de linha físico**;
- variações feitas **somente em \(\phi\)** (não na métrica).

---

## 1. Princípio físico fundamental

Um fluido perfeito macroscópico é caracterizado por:

- conservação do **número de partículas**;
- uma densidade de energia \(\rho(n)\), função da densidade numérica comóvel \(n\).

Ao variar \(\phi\), **não estamos criando nem destruindo partículas**.  
Estamos apenas mudando **como volumes e tempos físicos são medidos**.

👉 Portanto, a quantidade correta a manter fixa sob variações de \(\phi\) é o **conteúdo material**, não objetos que já embutem a medida geométrica.

---

## 2. A variável material correta: corrente densitizada

Defina a **métrica física**:

$$
\hat g_{\mu\nu} \equiv \phi^2 g_{\mu\nu},
\qquad
\sqrt{-\hat g} = \phi^4\sqrt{-g}.
$$

A corrente física de partículas é \(J^\mu\), mas a quantidade **conservada** é a corrente densitizada:

$$
\boxed{
\mathcal J^\mu \equiv \sqrt{-\hat g}\,J^\mu
}
$$

A conservação de partículas é:

$$
\partial_\mu \mathcal J^\mu = 0.
$$

**Heurística essencial:**  
Ao variar \(\phi\), mantemos \(\mathcal J^\mu\) fixa — isso garante que estamos descrevendo **o mesmo fluido físico**.

---

## 3. Construção da densidade numérica \(n\)

A densidade comóvel deve ser:

- um **escalar**;
- construída a partir da corrente;
- medida com a **métrica física**.

Definição correta:

$$
\boxed{
n
=
\frac{1}{\sqrt{-\hat g}}\,
\sqrt{\hat g_{\mu\nu}\,\mathcal J^\mu \mathcal J^\nu}
}
$$

Dependência em \(\phi\):

- \(\hat g_{\mu\nu} \propto \phi^2 \Rightarrow \sqrt{\hat g_{\mu\nu}\mathcal J^\mu\mathcal J^\nu}\propto \phi\);
- \(\sqrt{-\hat g}\propto \phi^4\).

Logo:

$$
\boxed{
n \propto \phi^{-3}
}
$$

Interpretação física:

> Se o comprimento escala como \(\phi\), o volume escala como \(\phi^3\);  
> portanto a densidade cai como \(1/\phi^3\).

---

## 4. Ação de matéria do fluido perfeito

A ação efetiva padrão é:

$$
\boxed{
S_m
=
-\int d^4x\,\sqrt{-\hat g}\;\rho(n)
}
$$

onde \(\rho(n)\) incorpora a equação de estado.

Essa ação **já depende de \(\phi\)** porque:
- a medida é \(\sqrt{-\hat g}\);
- \(n\) depende de \(\hat g\).

Nenhuma variação métrica foi usada.

---

## 5. Variação da ação com respeito a \(\phi\)

Variando:

$$
\delta S_m
=
-\int d^4x\left[
\delta\sqrt{-\hat g}\,\rho
+
\sqrt{-\hat g}\,\rho'(n)\,\delta n
\right]
$$

### 5.1 Variação da medida

$$
\delta\sqrt{-\hat g}
=
4\frac{\delta\phi}{\phi}\sqrt{-\hat g}
$$

### 5.2 Variação da densidade \(n\)

Como \(n\propto \phi^{-3}\):

$$
\boxed{
\delta n
=
-3\frac{n}{\phi}\,\delta\phi
}
$$

---

## 6. Resultado da variação

Substituindo:

$$
\delta S_m
=
-\int d^4x\,\sqrt{-\hat g}\,
\frac{\delta\phi}{\phi}
\left[
4\rho - 3n\rho'(n)
\right]
$$

Use a identidade termodinâmica do fluido perfeito:

$$
\boxed{
p = n\rho'(n) - \rho
}
\qquad\Rightarrow\qquad
n\rho'(n)=\rho+p
$$

Então:

$$
4\rho - 3(\rho+p) = \rho - 3p
$$

Logo:

$$
\boxed{
\delta S_m
=
-\int d^4x\,\sqrt{-\hat g}\,
\frac{\rho-3p}{\phi}\,\delta\phi
}
$$

---

## 7. Derivada funcional final

Forma covariante:

$$
\boxed{
\frac{1}{\sqrt{-\hat g}}
\frac{\delta S_m}{\delta\phi}
=
-\frac{\rho-3p}{\phi}
}
$$

Ou, reescrevendo com \(\sqrt{-\hat g}=\phi^4\sqrt{-g}\):

$$
\boxed{
\frac{1}{\sqrt{-g}}
\frac{\delta S_m}{\delta\phi}
=
-\phi^3(\rho-3p)
}
$$

---

## 8. Interpretação física

- O acoplamento de \(\phi\) à matéria é **proporcional ao traço** do tensor energia–momento:
  $$
  T = \rho - 3p
  $$
- Radiação (\(p=\rho/3\)) **não fonteia** \(\phi\);
- Matéria fria (\(p\approx 0\)) fonteia fortemente;
- Vácuo (\(p=-\rho\)) é fonte máxima.

---

## 9. Conclusão

- O termo espúrio \(5\rho\) surge ao segurar a variável errada fixa;
- A variável física correta é a **corrente densitizada** \(\mathcal J^\mu\);
- Com essa escolha, o acoplamento em Lyra fica coerente e controlado.



In [4]:
_, a, b, g, d, e, m, n = st.index("empty alpha beta gamma delta epsilon mu nu")


phi = st.set_scale(sp.Function('phi')(r))

nabla_phi = st.nabla(phi)

# nabla_phi[-m] * nabla_phi[+m]


st.nabla(phi * nabla_phi)[-m,+m]

In [5]:
nabla_nabla_phi[-m, +m]

In [6]:
sp.expand(sp.simplify(nabla_nabla_phi[+m, -m]))

-Derivative(phi(r), (r, 2))/phi(r)**2 - 2*Derivative(phi(r), r)**2/phi(r)**3 - 2*Derivative(phi(r), r)/(r*phi(r)**2)

In [7]:
c1, c2 = sp.symbols('c1 c2', real=True)

L = (sp.sqrt(-st.detg) * phi**4 * (
    0
    # + c1 * st.scalar_curvature
    # + c2 *  nabla_phi[-m] * nabla_phi[+m]
    # + c1 * sp.expand(sp.simplify(nabla_nabla_phi[+m, -m]))
    
)).subs(th,sp.pi/2)

term_0 = sp.diff(L, phi)
term_1 = sp.diff(sp.diff(L, sp.diff(phi, r)), r)
term_2 = sp.diff(sp.diff(L, sp.diff(phi, r, 2)), r, 2)

display(sp.simplify(term_0 - term_1 + term_2))

# field_eq = sp.Eq(sp.simplify(term_0 - term_1 + term_2), 0)

# display(field_eq)

# expr = sp.factor(field_eq.lhs)

# expr_nt = sp.cancel(expr / phi)

# eq_nt = sp.Eq(sp.expand(sp.simplify(expr_nt)), 0)

# display(eq_nt)

# sp.dsolve(eq_nt, phi.expr)

0

Rodei
```
c1, c2 = sp.symbols('c1 c2', real=True)

L = (sp.sqrt(-st.detg) * phi**4 * (
    0
    # + c1 * st.scalar_curvature
    # + c2 *  nabla_phi[-m] * nabla_phi[+m]
    # + c1 * sp.expand(sp.simplify(nabla_nabla_phi[+m, -m]))

    + st.nabla(phi**2 * nabla_phi)[-m,+m]
    
)).subs(th,sp.pi/2)

term_0 = sp.diff(L, phi)
term_1 = sp.diff(sp.diff(L, sp.diff(phi, r)), r)
term_2 = sp.diff(sp.diff(L, sp.diff(phi, r, 2)), r, 2)

display(sp.simplify(term_0 - term_1 + term_2))
```

e tomeio erro:
```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[10], line 9
      1 c1, c2 = sp.symbols('c1 c2', real=True)
      3 L = (sp.sqrt(-st.detg) * phi**4 * (
      4     0
      5     # + c1 * st.scalar_curvature
      6     # + c2 *  nabla_phi[-m] * nabla_phi[+m]
      7     # + c1 * sp.expand(sp.simplify(nabla_nabla_phi[+m, -m]))
      8 
----> 9     + st.nabla(phi**2 * nabla_phi)[-m,+m]
     10 
     11 )).subs(th,sp.pi/2)
     13 term_0 = sp.diff(L, phi)
     14 term_1 = sp.diff(sp.diff(L, sp.diff(phi, r)), r)

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/core/decorators.py:243, in _SympifyWrapper.make_wrapped.<locals>._func(self, other)
    241 if not hasattr(other, '_op_priority'):
    242     try:
--> 243         other = sympify(other, strict=True)
    244     except SympifyError:
    245         return retval

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/core/sympify.py:443, in sympify(a, locals, convert_xor, strict, rational, evaluate)
    441 _sympy_ = getattr(a, "_sympy_", None)
    442 if _sympy_ is not None:
--> 443     return a._sympy_()
    445 if not strict:
    446     # Put numpy array conversion _before_ float/int, see
    447     # <https://github.com/sympy/sympy/issues/13924>.
    448     flat = getattr(a, "flat", None)

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:601, in Tensor._sympy_(self)
    600 def _sympy_(self):
--> 601     return self._as_scalar()

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:567, in Tensor._as_scalar(self)
    565 def _as_scalar(self):
    566     if self.rank != 0:
--> 567         raise TypeError("Operacao escalar so e valida para tensores de rank 0.")
    568     return sp.sympify(self.components[()])

TypeError: Operacao escalar so e valida para tensores de rank 0.
```  Por que o framework está dando tanto erro assim?

In [76]:
sp.expand(sp.simplify(L))

-c1*r**2*phi(r)**2*Derivative(phi(r), (r, 2)) - 2*c1*r**2*phi(r)*Derivative(phi(r), r)**2 - 2*c1*r*phi(r)**2*Derivative(phi(r), r) - c2*r**2*phi(r)**2*Derivative(phi(r), r)**2

In [72]:
def strip_hold_mathematica(s: str) -> str:
    out = []
    i = 0
    n = len(s)

    while i < n:
        if s.startswith("Hold[", i):
            i += 5  # pula "Hold["
            depth = 1
            start = i

            while i < n and depth > 0:
                if s[i] == "[":
                    depth += 1
                elif s[i] == "]":
                    depth -= 1
                i += 1

            # conteúdo dentro de Hold[ ... ]  (sem o último ']')
            inner = s[start:i-1]
            out.append(inner)
        else:
            out.append(s[i])
            i += 1

    return "".join(out)

s = sp.mathematica_code(eq_nt)
print(strip_hold_mathematica(s))


2*c2*r^2*phi[r]*D[phi[r], {r, 2}] + 2*c2*r^2*D[phi[r], r]^2 + 4*c2*r*phi[r]*D[phi[r], r] == 0


In [64]:
sp.expand(L)

-6*c1*r**2*phi(r)*Derivative(phi(r), (r, 2)) - 12*c1*r*phi(r)*Derivative(phi(r), r) - c2*r**2*phi(r)**2*Derivative(phi(r), r)**2

In [48]:
lhs = eq_nt.lhs.replace(
    lambda x: isinstance(x, sp.Derivative),
    lambda x: x.doit()
)

rhs = eq_nt.rhs.replace(
    lambda x: isinstance(x, sp.Derivative),
    lambda x: x.doit()
)

print(sp.mathematica_code(sp.Eq(lhs, rhs)))

2*r*(r*phi[r]*Hold[D[phi[r], {r, 2}]] + r*Hold[D[phi[r], r]]^2 + 2*phi[r]*Hold[D[phi[r], r]]) == 0


In [49]:
sp.mathematica_code(eq_nt)

'2*r*(r*phi[r]*Hold[D[phi[r], {r, 2}]] + r*Hold[D[phi[r], r]]^2 + 2*phi[r]*Hold[D[phi[r], r]]) == 0'

Eu rodei `print(sp.mathematica_code(eq_nt))` e o resultado foi

```wolfram
2*r*(r*phi[r]*Hold[D[phi[r], {r, 2}]] + r*Hold[D[phi[r], r]]^2 + 2*phi[r]*Hold[D[phi[r], r]]) == 0
```

em vez de:

```wolfram
2*r*(r*phi[r]*D[phi[r], {r, 2}] + r*D[phi[r], r]^2 + 
    2*phi[r]*D[phi[r], r]) == 0
```

In [6]:
from lyra_geometry import Tensor
def to_expr(x):
    return x.expr if isinstance(x, Tensor) else x

L_expr = to_expr(L)
phi_expr = to_expr(phi)

term_0 = sp.diff(L_expr, phi_expr)
term_1 = sp.diff(sp.diff(L_expr, sp.diff(phi_expr, r)), r)
term_2 = sp.diff(sp.diff(L_expr, sp.diff(phi_expr, r, 2)), r, 2)

field_eq = sp.simplify(term_0 - term_1 + term_2)
sp.dsolve(field_eq, phi_expr)


NameError: name 'L' is not defined

In [32]:
type(phi)

lyra_geometry.Tensor

In [5]:


sp.dsolve(field_eq, phi, r)


2*r**2*phi(r)**2*Derivative(phi(r), (r, 2)) + 2*r**2*phi(r)*Derivative(phi(r), r)**2 + 4*r*phi(r)**2*Derivative(phi(r), r)

In [20]:
field_eq.args

(2*r**2*phi(r)*Derivative(phi(r), r)**2,
 2*r**2*phi(r)**2*Derivative(phi(r), (r, 2)),
 4*r*phi(r)**2*Derivative(phi(r), r))

In [23]:
phi

In [24]:
2*r**2*phi*Derivative(phi, r)**2

NameError: name 'Derivative' is not defined

In [10]:
sp.dsolve(sp.Eq(field_eq,0), phi, r)

AttributeError: 'Tensor' object has no attribute 'args'

In [7]:

print("--- Lagrangian ---")
# Expand serve para distribuir termos, similar ao Expand do Mathematica
print(expand(L))

# 3. Calculando a Equação de Campo (Euler-Lagrange de ordem superior)
# Termo 0: dL/dphi
term_0 = diff(L, phi)

# Termo 1: - d/dr ( dL/dphi' )
# diff(phi, r) é a primeira derivada
term_1 = diff(diff(L, diff(phi, r)), r)

# Termo 2: + d^2/dr^2 ( dL/dphi'' )
# diff(phi, r, 2) é a segunda derivada
term_2 = diff(diff(L, diff(phi, r, 2)), r, 2)

# Equação completa: Termo 0 - Termo 1 + Termo 2
Eq_phi = term_0 - term_1 + term_2

print("\n--- Field Equation for phi ---")
# FullSimplify do Mathematica equivale aproximadamente a simplify() no SymPy
field_eq = simplify(expand(Eq_phi))
print(field_eq)

-r**2*phi(r)**2*Abs(sin(theta))*Derivative(phi(r), r)**2

In [13]:
st.scalar_curvature


In [24]:
nabla_G = st.nabla(G, deriv_position='prepend')

In [32]:
sp.expand(sp.simplify(nabla_G[-𝛼,-𝛽,-𝜇](1, 0, 0)))

2*Derivative(phi(r), (r, 3))/phi(r)**4 - 8*Derivative(phi(r), r)*Derivative(phi(r), (r, 2))/phi(r)**5 + 4*Derivative(phi(r), r)**3/phi(r)**6 + 4*Derivative(phi(r), (r, 2))/(r*phi(r)**4) - 12*Derivative(phi(r), r)**2/(r*phi(r)**5) - 4*Derivative(phi(r), r)/(r**2*phi(r)**4)

In [34]:
print(sp.latex(sp.expand(sp.simplify(nabla_G[-𝛼,-𝛽,-𝜇](1, 1, 1)))))

- \frac{6 \frac{d}{d r} \phi{\left(r \right)} \frac{d^{2}}{d r^{2}} \phi{\left(r \right)}}{\phi^{5}{\left(r \right)}} + \frac{12 \left(\frac{d}{d r} \phi{\left(r \right)}\right)^{3}}{\phi^{6}{\left(r \right)}} - \frac{4 \frac{d^{2}}{d r^{2}} \phi{\left(r \right)}}{r \phi^{4}{\left(r \right)}} + \frac{12 \left(\frac{d}{d r} \phi{\left(r \right)}\right)^{2}}{r \phi^{5}{\left(r \right)}} + \frac{4 \frac{d}{d r} \phi{\left(r \right)}}{r^{2} \phi^{4}{\left(r \right)}}


Putz, então parece que temos alguma inconsistência teórico porque o tensor de Einstein aparentemente não é conservado via derivadas covariantes de Lyra.
Abaixo está o resutado de `sp.expand(sp.simplify(nabla_G[-𝛼,-𝛽,-𝜇](1, 1, 1)))`
$$
- \frac{6 \frac{d}{d r} \phi{\left(r \right)} \frac{d^{2}}{d r^{2}} \phi{\left(r \right)}}{\phi^{5}{\left(r \right)}} + \frac{12 \left(\frac{d}{d r} \phi{\left(r \right)}\right)^{3}}{\phi^{6}{\left(r \right)}} - \frac{4 \frac{d^{2}}{d r^{2}} \phi{\left(r \right)}}{r \phi^{4}{\left(r \right)}} + \frac{12 \left(\frac{d}{d r} \phi{\left(r \right)}\right)^{2}}{r \phi^{5}{\left(r \right)}} + \frac{4 \frac{d}{d r} \phi{\left(r \right)}}{r^{2} \phi^{4}{\left(r \right)}}
$$

Alguma chance disso ser zero?

Rodei 
```
from lyra_geometry import Tensor
def to_expr(x):
    return x.expr if isinstance(x, Tensor) else x

L_expr = to_expr(L)
phi_expr = to_expr(phi)

term_0 = sp.diff(L_expr, phi_expr)
term_1 = sp.diff(sp.diff(L_expr, sp.diff(phi_expr, r)), r)
term_2 = sp.diff(sp.diff(L_expr, sp.diff(phi_expr, r, 2)), r, 2)

field_eq = sp.simplify(term_0 - term_1 + term_2)
sp.dsolve(field_eq, phi_expr, r)
```

e o resultado foi
```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[26], line 13
     10 term_2 = sp.diff(sp.diff(L_expr, sp.diff(phi_expr, r, 2)), r, 2)
     12 field_eq = sp.simplify(term_0 - term_1 + term_2)
---> 13 sp.dsolve(field_eq, phi_expr, r)

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/solvers/ode/ode.py:605, in dsolve(eq, func, hint, simplify, ics, xi, eta, x0, n, **kwargs)
    602 given_hint = hint  # hint given by the user
    604 # See the docstring of _desolve for more details.
--> 605 hints = _desolve(eq, func=func,
    606     hint=hint, simplify=True, xi=xi, eta=eta, type='ode', ics=ics,
    607     x0=x0, n=n, **kwargs)
    608 eq = hints.pop('eq', eq)
    609 all_ = hints.pop('all', False)

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/solvers/deutils.py:209, in _desolve(eq, func, hint, ics, simplify, prep, **kwargs)
    205 # Magic that should only be used internally.  Prevents classify_ode from
    206 # being called more than it needs to be by passing its results through
    207 # recursive calls.
    208 if kwargs.get('classify', True):
--> 209     hints = classifier(eq, func, dict=True, ics=ics, xi=xi, eta=eta,
    210     n=terms, x0=x0, hint=hint, prep=prep)
    212 else:
    213     # Here is what all this means:
    214     #
   (...)    223     #          hint.
    224     # order:   The order of the DE, as determined by ode_order().
    225     hints = kwargs.get('hint',
    226                        {'default': hint,
    227                         hint: kwargs['match'],
    228                         'order': kwargs['order']})

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/solvers/ode/ode.py:1030, in classify_ode(eq, func, dict, ics, prep, xi, eta, n, **kwargs)
   1027 # Used when dsolve is called without an explicit hint.
   1028 # We exit early to return the first valid match
   1029 early_exit = (user_hint=='default')
-> 1030 user_hint = user_hint.removesuffix('_Integral')
   1031 user_map = solver_map
   1032 # An explicit hint has been given to dsolve
   1033 # Skip matching code for other hints

AttributeError: 'Symbol' object has no attribute 'removesuffix'
```

Poderia conferir o $\nabla_{0} G_{01}$. Pra mim aqui deu:
$$
\nabla_{0} G_{01} = \frac{2 \left(- \phi{\left(r \right)} \frac{d^{2}}{d r^{2}} \phi{\left(r \right)} + 2 \left(\frac{d}{d r} \phi{\left(r \right)}\right)^{2}\right) \frac{d}{d r} \phi{\left(r \right)}}{\phi^{6}{\left(r \right)}}
$$

Saiba que $G[-𝛼,-𝛽](0, 1)=0$

In [29]:
G[-𝛼,-𝛽](0, 1)

0

In [28]:
print(sp.latex(nabla_G[-𝛼,-𝛽,-𝜇](0, 0, 1)))

\frac{2 \left(- \phi{\left(r \right)} \frac{d^{2}}{d r^{2}} \phi{\left(r \right)} + 2 \left(\frac{d}{d r} \phi{\left(r \right)}\right)^{2}\right) \frac{d}{d r} \phi{\left(r \right)}}{\phi^{6}{\left(r \right)}}


In [ ]:
from sympy import symbols, Function, diff, sqrt, simplify, expand, pi, init_printing

# Habilita "pretty printing" para saída mais legível no terminal/Jupyter
init_printing()

# 1. Definições Iniciais
r = symbols('r')
theta = symbols('theta')

# Definindo a função de campo phi(r)
phi = Function('phi')(r)

# --- IMPORTANTE ---
# O Mathematica 'sabe' implicitamente que g e R dependem de phi, phi' e phi''.
# No SymPy, você PRECISA definir explicitamente o que são g e R em termos de phi
# para que as derivadas não sejam zero.
# Abaixo, coloco placeholders genéricos. Substitua pelas suas expressões reais de FLRW/Lyra.

# Exemplo genérico (substitua isso pelas suas equações reais de g e R)
# g = -r**2 * phi  # Exemplo fictício
# R = diff(phi, r, 2) + diff(phi, r)**2 # Exemplo fictício

# Se você ainda não tem as expressões e quer apenas a estrutura simbólica abstrata:
g = Function('g')(r)
R = Function('R')(r)
# Nota: Se usar a forma abstrata acima, as derivadas em relação a phi' serão zero
# a menos que você substitua g e R antes de derivar.

# 2. Definindo a Lagrangiana
# L = Sqrt[-g] * Phi^4 * R
# Assumindo que CapitalPhi do seu código é o próprio campo phi
L = sqrt(-g) * phi**4 * R

# Substituição de theta -> pi/2 (aplicar se houver theta na expressão)
L = L.subs(theta, pi/2)

print("--- Lagrangian ---")
# Expand serve para distribuir termos, similar ao Expand do Mathematica
print(expand(L))

# 3. Calculando a Equação de Campo (Euler-Lagrange de ordem superior)
# Termo 0: dL/dphi
term_0 = diff(L, phi)

# Termo 1: - d/dr ( dL/dphi' )
# diff(phi, r) é a primeira derivada
term_1 = diff(diff(L, diff(phi, r)), r)

# Termo 2: + d^2/dr^2 ( dL/dphi'' )
# diff(phi, r, 2) é a segunda derivada
term_2 = diff(diff(L, diff(phi, r, 2)), r, 2)

# Equação completa: Termo 0 - Termo 1 + Termo 2
Eq_phi = term_0 - term_1 + term_2

print("\n--- Field Equation for phi ---")
# FullSimplify do Mathematica equivale aproximadamente a simplify() no SymPy
field_eq = simplify(expand(Eq_phi))
print(field_eq)

In [16]:

rmn = st.g[+𝛼,+𝜆] * st.riemann[-𝛼,-𝛽,-𝜇,-𝜈]

In [15]:
sp.expand(st.riemann[+𝛼,-𝛽,-𝜇,-𝜈](0,1,0,1))

-Derivative(alpha(r), (r, 2))/(2*alpha(r)*phi(r)**2) - Derivative(alpha(r), r)*Derivative(phi(r), r)/(2*alpha(r)*phi(r)**3) - Derivative(alpha(r), r)*Derivative(beta(r), r)/(4*alpha(r)*beta(r)*phi(r)**2) + Derivative(alpha(r), r)**2/(4*alpha(r)**2*phi(r)**2)

In [17]:
print(sp.latex(sp.expand(st.riemann[+𝛼,-𝛽,-𝜇,-𝜈](0,1,0,1))))

- \frac{\frac{d^{2}}{d r^{2}} \alpha{\left(r \right)}}{2 \alpha{\left(r \right)} \phi^{2}{\left(r \right)}} - \frac{\frac{d}{d r} \alpha{\left(r \right)} \frac{d}{d r} \phi{\left(r \right)}}{2 \alpha{\left(r \right)} \phi^{3}{\left(r \right)}} - \frac{\frac{d}{d r} \alpha{\left(r \right)} \frac{d}{d r} \beta{\left(r \right)}}{4 \alpha{\left(r \right)} \beta{\left(r \right)} \phi^{2}{\left(r \right)}} + \frac{\left(\frac{d}{d r} \alpha{\left(r \right)}\right)^{2}}{4 \alpha^{2}{\left(r \right)} \phi^{2}{\left(r \right)}}


Tentei isso `st.riemann[-𝛼,+𝛽,-𝜇,-𝜈]` mas deu um erro:

```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[12], line 1
----> 1 st.riemann[-𝛼,+𝛽,-𝜇,-𝜈]

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:626, in Tensor.__getitem__(self, indices)
    624     else:
    625         down[i] = idx.label
--> 626 indexed = self.idx(up=up, down=down)
    627 labels = indexed.labels
    628 if len(set(labels)) != len(labels):

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:806, in Tensor.idx(self, up, down)
    803         target_sig.append(D)
    804         labels.append(self.space._next_label() if down_i is NO_LABEL else down_i)
--> 806 A = self.as_signature(tuple(target_sig), simplify=False)
    807 return IndexedTensor(self, A, tuple(target_sig), labels)

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:734, in Tensor.as_signature(self, target_signature, simplify)
    732     sig_cur[pos] = U
    733 elif have is U and want is D:
--> 734     A = self._lower_at(A, pos)
    735     sig_cur[pos] = D
    736 else:

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:714, in Tensor._lower_at(self, A, pos)
    712 if self.space.metric is None:
    713     raise ValueError("Metric nao definida para descer indices.")
--> 714 TP = sp.tensorproduct(self.space.metric, A)
    715 C = sp.tensorcontraction(TP, (1, pos + 2))
    716 return self._move_front_axis_to(C, pos)

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/tensor/array/arrayop.py:79, in tensorproduct(*args)
     76 a, b = map(_arrayfy, args)
     78 if not isinstance(a, NDimArray) or not isinstance(b, NDimArray):
---> 79     return a*b
     81 if isinstance(a, SparseNDimArray) and isinstance(b, SparseNDimArray):
     82     lp = len(b)

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/tensor/array/ndim_array.py:445, in NDimArray.__rmul__(self, other)
    442 if isinstance(other, (Iterable, NDimArray, MatrixBase)):
    443     raise ValueError("scalar expected, use tensorproduct(...) for tensorial product")
--> 445 other = sympify(other)
    446 if isinstance(self, SparseNDimArray):
    447     if other.is_zero:

File ~/.pyenv/versions/3.13.5/lib/python3.13/site-packages/sympy/core/sympify.py:443, in sympify(a, locals, convert_xor, strict, rational, evaluate)
    441 _sympy_ = getattr(a, "_sympy_", None)
    442 if _sympy_ is not None:
--> 443     return a._sympy_()
    445 if not strict:
    446     # Put numpy array conversion _before_ float/int, see
    447     # <https://github.com/sympy/sympy/issues/13924>.
    448     flat = getattr(a, "flat", None)

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:582, in Tensor._sympy_(self)
    581 def _sympy_(self):
--> 582     return self._as_scalar()

File ~/Documentos/git/lyra_geometry/lyra_geometry.py:560, in Tensor._as_scalar(self)
    558 def _as_scalar(self):
    559     if self.rank != 0:
--> 560         raise TypeError("Operacao escalar so e valida para tensores de rank 0.")
    561     return sp.sympify(self.components[()])

TypeError: Operacao escalar so e valida para tensores de rank 0.
```

In [5]:
_, a, b, m, n = st.index("empty a b m n")

st.riemann[+a, -b, -m, -n](0, 1, 0, 1)


(Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - Derivative(nu(r), r)**2/4 - Derivative(nu(r), (r, 2))/2)*exp(-2*lambda(r))*exp(-nu(r))

poderia fazer com que todo tensor indexado tivesse o método .fmt() que faz isso:

```python

def fmt(self, expr):
    return sp.expand(sp.simplify(expr))

```

In [37]:
import lyra_geometry as pl

pl.greek("alpha")


Exemplo de torsao (defina `eps`, `s`, `v` e `st` antes de usar).


In [ ]:
import sympy as sp
import lyra_geometry as pl

# Acesso a componentes apos indexacao

x, y = sp.symbols('x y')
st = pl.SpaceTime(coords=(x, y))

R = st.tensor.generic('R', (pl.U, pl.D, pl.D, pl.D))
a, b, c, d = st.index('a b c d')

# indexacao Einstein e acesso direto aos componentes
R[+a, -b, -c, -d](0, 1, 0, 1)
# forma explicita
R[+a, -b, -c, -d].get(0, 1, 0, 1)


In [3]:
# (ja mostrado acima)


AttributeError: 'IndexedTensor' object has no attribute 'comp'

In [3]:
def fmt(expr):
    return sp.expand(sp.simplify(expr))

In [2]:
st.update()

In [4]:
fmt(st.ricci(pl.d, pl.d).comp[0, 0])


s_r(r)**2*exp(f1(r))*exp(f2(r))/2 + v_r(r)**2*exp(f1(r))*exp(-f2(r))/9 + v_r(r)*exp(f1(r))*exp(-f2(r))*Derivative(f2(r), r)/(6*phi(r)) - exp(f1(r))*exp(-f2(r))*Derivative(v_r(r), r)/(3*phi(r)) + v_r(r)*exp(f1(r))*exp(-f2(r))*Derivative(phi(r), r)/(3*phi(r)**2) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)**2/(4*phi(r)**2) + exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)*Derivative(f2(r), r)/(4*phi(r)**2) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), (r, 2))/(2*phi(r)**2) + exp(f1(r))*exp(-f2(r))*Derivative(f2(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) - exp(f1(r))*exp(-f2(r))*Derivative(phi(r), (r, 2))/phi(r)**3 + 2*exp(f1(r))*exp(-f2(r))*Derivative(phi(r), r)**2/phi(r)**4 - 2*v_r(r)*exp(f1(r))*exp(-f2(r))/(3*r*phi(r)) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)/(r*phi(r)**2) - 2*exp(f1(r))*exp(-f2(r))*Derivative(phi(r), r)/(r*phi(r)**3)

In [7]:
fmt(st.ricci(pl.u, pl.u).comp[1, 1])


s_t(r)**2*exp(f1(r))*exp(-f2(r))/2 + v_t(r)**2*exp(-f1(r))*exp(-f2(r))/9 + v_r(r)*exp(-2*f2(r))*Derivative(f1(r), r)/(6*phi(r)) + exp(-2*f2(r))*Derivative(f1(r), r)**2/(4*phi(r)**2) - exp(-2*f2(r))*Derivative(f1(r), r)*Derivative(f2(r), r)/(4*phi(r)**2) + exp(-2*f2(r))*Derivative(f1(r), (r, 2))/(2*phi(r)**2) + exp(-2*f2(r))*Derivative(f1(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) + 2*v_r(r)*exp(-2*f2(r))/(3*r*phi(r)) - exp(-2*f2(r))*Derivative(f2(r), r)/(r*phi(r)**2) + 2*exp(-2*f2(r))*Derivative(phi(r), r)/(r*phi(r)**3)

In [15]:
a = st.index("a")

fmt(st.contract(st.ricci.idx((a, None), (None, a))).expr)

3*s_r(r)**2*exp(f2(r))/2 - 3*s_t(r)**2*exp(f1(r))/2 + v_r(r)**2*exp(-f2(r))/3 - v_t(r)**2*exp(-f1(r))/3 - v_r(r)*exp(-f2(r))*Derivative(f1(r), r)/(2*phi(r)) + v_r(r)*exp(-f2(r))*Derivative(f2(r), r)/(2*phi(r)) - exp(-f2(r))*Derivative(v_r(r), r)/phi(r) + v_r(r)*exp(-f2(r))*Derivative(phi(r), r)/phi(r)**2 - exp(-f2(r))*Derivative(f1(r), r)**2/(2*phi(r)**2) + exp(-f2(r))*Derivative(f1(r), r)*Derivative(f2(r), r)/(2*phi(r)**2) - exp(-f2(r))*Derivative(f1(r), (r, 2))/phi(r)**2 - 3*exp(-f2(r))*Derivative(f1(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) + 3*exp(-f2(r))*Derivative(f2(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) - 3*exp(-f2(r))*Derivative(phi(r), (r, 2))/phi(r)**3 + 6*exp(-f2(r))*Derivative(phi(r), r)**2/phi(r)**4 - 2*v_r(r)*exp(-f2(r))/(r*phi(r)) - 2*exp(-f2(r))*Derivative(f1(r), r)/(r*phi(r)**2) + 2*exp(-f2(r))*Derivative(f2(r), r)/(r*phi(r)**2) - 6*exp(-f2(r))*Derivative(phi(r), r)/(r*phi(r)**3) + 2/(r**2*phi(r)**2) - 2*exp(-f2(r))/(r**2*phi(r)**2)

In [17]:
Rmn = st.riemann
Rcc = st.ricci
R = st.scalar_curvature

Agora, eu gostaria de mudar a sintaxe de:

Rmn = st.riemann

```python
Rmn.idx((a, None, None, None), (None, a, b, c))
```

para

```python
Rmn.up(a,None,None,None).down(None,a,b,c)
```

Além disso, faça esses dois métodos completarem a direito os indices nao fornecidos. Por exemplo:


```python
Rmn.up(a).down(None,a,b,c)
```

Nesse caso acima ficaria:

$
R^{a}_{\;abc}
$



Exemplo: `dT = st.nabla(T)` e acesso via `dT.comp[i, j, k]`.


In [8]:
st.gamma[0,0,0]

ValueError: Conexao nao definida.

# Sandbox

In [2]:
from lyra_geometry import *
import sympy as sp

# -- Metrics --------

print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)
dim = 4

lamb = sp.Function('lambda')
nu = sp.Function('nu')

g = sp.diag(sp.exp(nu(r)), -sp.exp(lamb(r)), -r**2, -r**2*sp.sin(th)**2)

# -- SpaceTime ----------
st = SpaceTime(dim=4, coords=x, metric=g)

Ricc = st.ricci
R = st.scalar_curvature
_, a = st.index("empty a")

for i in range(3):
    display(
        sp.Eq(sp.expand(st.einstein(d, d).comp[i, i]), 0)
    )


Definindo metrica


Eq(-exp(-lambda(r))*exp(nu(r))*Derivative(lambda(r), r)/r - exp(nu(r))/r**2 + exp(-lambda(r))*exp(nu(r))/r**2, 0)

Eq(-Derivative(nu(r), r)/r + exp(lambda(r))/r**2 - 1/r**2, 0)

Eq(r**2*exp(-lambda(r))*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*Derivative(nu(r), r)/2, 0)

In [10]:
for i in range(3):
    display(
        sp.Eq(sp.expand(Ricc(d, d).comp[i, i] - sp.Rational(1, 2) * st.g(d, d).comp[i, i] * R), 0)
    )


Eq(-exp(-lambda(r))*exp(nu(r))*Derivative(lambda(r), r)/r - exp(nu(r))/r**2 + exp(-lambda(r))*exp(nu(r))/r**2, 0)

Eq(-Derivative(nu(r), r)/r + exp(lambda(r))/r**2 - 1/r**2, 0)

Eq(r**2*exp(-lambda(r))*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*Derivative(nu(r), r)/2, 0)

In [4]:
R_contract = st.contract(Ricc.idx((a, None), (None, a)))
R_contract


In [1]:
from lyra_geometry import *
import sympy as sp

# -------------------------
# Coords / functions
# -------------------------
t, r, th, vph = sp.symbols("t r theta varphi", real=True)
x = (t, r, th, vph)
dim = 4

a = sp.Function("a")   # scale factor a(t)
k = sp.symbols("k", real=True)  # curvature constant (often -1,0,+1)

# -------------------------
# FLRW metric: diag(+,-,-,-)
# -------------------------
g = sp.diag(
    1,
    -a(t)**2 / (1 - k*r**2),
    -a(t)**2 * r**2,
    -a(t)**2 * r**2 * sp.sin(th)**2
)

st = SpaceTime(dim=4, coords=x, metric=g)



# -------------------------
# If lyra_geometry already exposes curvature tensors, use them.
# (Names may vary depending on your library.)
# -------------------------
try:
    # Example-style (adjust if your API uses different names)
    Ricc = st.ricci            # rank-2 tensor
    R = st.scalar_curvature    # scalar
    Ein = st.einstein          # rank-2 tensor

    # show Einstein equations G_{μν} = 0 (vacuum) or G_{μν} = 8πT_{μν}
    for i in range(4):
        print(f"G[{i},{i}] =")
        display(sp.simplify(Ein(d, d).comp[i, i]))

except Exception as e:
    print("Não achei st.ricci / st.einstein nessa versão do lyra_geometry.")
    print("Erro:", e)
    print("Use a versão 2 abaixo (curvatura via from_function).")


G[0,0] =


3*(-k - Derivative(a(t), t)**2)/a(t)**2

G[1,1] =


(-k - 2*a(t)*Derivative(a(t), (t, 2)) - Derivative(a(t), t)**2)/(k*r**2 - 1)

G[2,2] =


r**2*(k + 2*a(t)*Derivative(a(t), (t, 2)) + Derivative(a(t), t)**2)

G[3,3] =


r**2*(k + 2*a(t)*Derivative(a(t), (t, 2)) + Derivative(a(t), t)**2)*sin(theta)**2

Notas antigas sobre a API (mantidas como rascunho).


Na verdade, queria melhorar as abstrações. Da forma como está agora:

- A métrica não é um tensor
- Escalares não são tensores
- Ter alguma forma de fazer contrações


Pensei em ter um elemento que abstraia o espaço-tempo. Manifold? Não sei

Dai, deveriamos ter métodos set_scale. Se não passar, usa-se phi = sp.Function('phi')(r)

um método set metric
set torsion
set metrica compatibility

e uma forma de fazer contração de indices


In [ ]:
M.contract(
    
)

In [1]:
from lyra_geometry import *

t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)

st = SpaceTime(
    dim=4,
    coords=x,
    metric=sp.diag(
        sp.exp(sp.Function('f1')(r)),
        -sp.exp(sp.Function('f2')(r)),
        -r**2,
        -r**2*sp.sin(th)**2
    ),
)

# defaults
st.scale      # Tensor rank 0 (phi = 1)
st.torsion    # tau = 0
st.nonmetricity  # M = 0

st.set_scale(sp.Function('phi')(r))

A = st.tensor.from_function(lambda i, j: sp.Function('A')(r), signature=(d, d), name="A")
B = st.tensor.from_function(lambda i, j, k: sp.Function('B')(r), signature=(d, d, d), name="B")

_, a, b, c = st.index("empty a b c")

# índices por posição (up/down por posição do rank)
res1 = st.contract(A.idx(up=(a, _), down=(_, a)))

# contração entre dois tensores
res2 = st.contract(
    A.idx(up=(a, b), down=(_, _)),
    B.idx(up=(_, _, _), down=(a, b, c))
)

# # parsing por string
# res3 = st.eval_contract("A^{a,}_{ ,a}")
# res4 = st.eval_contract("A^{a,b} B_{a,b,c}")


In [2]:
st.scale

AttributeError: 'ImmutableDenseNDimArray' object has no attribute '_repr_html_'

# Novos exemplos: indexacao segura e estrategias

Esta secao mostra a nova indexacao Einstein via `u()`/`d()` e o encadeamento de contracoes.


In [1]:
from lyra_geometry import *
import sympy as sp

x, y = sp.symbols('x y')
st = SpaceTime(coords=(x, y))

a, b, c = st.index('a b c')
T = st.generic('T', (u, d))
g = st.generic('g', (u, u))
V = st.generic('V', (d,))


In [2]:
# Contracao segura com indexacao Einstein
expr1 = T[u(a), d(b)] * g[u(b), u(c)]
expr1.signature


TypeError: '>=' not supported between instances of 'UpIndex' and 'Integer'

In [3]:
# Encadeamento de contracoes (A * B * C)
expr2 = T[u(a), d(b)] * g[u(b), u(c)] * V[d(c)]
expr2.signature


TypeError: '>=' not supported between instances of 'UpIndex' and 'Integer'

In [4]:
# Parse de string ainda funciona, agora com indices mistos em ordem horizontal
T3 = st.generic('T3', (u, d, u))
S3 = st.generic('S3', (d, u, d))
expr3 = st.eval_contract('T3^{a}{}_{b}{}^{c} S3_{a}{}^{b}{}_{c}')
expr3.signature


()

In [5]:
# Exemplo de estrategia de conexao fixa (Gamma = 0)
Gamma0 = sp.ImmutableDenseNDimArray([0] * (2**3), (2, 2, 2))
st2 = SpaceTime(coords=(x, y), connection_strategy=FixedConnectionStrategy(Gamma0))
st2.update(include=['connection'])
st2.gamma.components


[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]